In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras import layers

In [2]:
file_path = "/Users/kiyoshitakeuchi/Desktop/Machine Learning/house-prices-advanced-regression-techniques/"
file_name = "train.csv"
test_file_name = "test.csv"

In [3]:
train_set = pd.read_csv(file_path + file_name)
train_set.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,1,2008,WD,Normal,118000


In [4]:
label = "SalePrice"
test_set = pd.read_csv(file_path + test_file_name)
test_set.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
5,1466,60,RL,75.0,10000,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal
6,1467,20,RL,NaN,7980,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,GdPrv,Shed,500,3,2010,WD,Normal
7,1468,60,RL,63.0,8402,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal
8,1469,20,RL,85.0,10176,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2010,WD,Normal
9,1470,20,RL,70.0,8400,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,4,2010,WD,Normal


In [5]:
def remove_nan(data_frame): 
    mean = data_frame.select_dtypes(include = 'number').median()
    mean = mean.to_dict()
    data_frame.fillna(value=mean, inplace=True)
    data_frame.fillna(value="?", inplace=True)
    return

remove_nan(test_set)
remove_nan(train_set)

In [6]:
categorical = []

for name, column in train_set.items():
        dtype = column.dtype
        if dtype == "int64" and len(np.unique(train_set[name])) < 20:
            categorical.append(name)
print(categorical)

ncf_dict = {name:"str" for name in categorical}
test_set = test_set.astype(ncf_dict)

['MSSubClass', 'OverallQual', 'OverallCond', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'PoolArea', 'MoSold', 'YrSold']


In [7]:
items_to_remove = ['YrSold', 'Fireplaces', 'GarageCars', 'TotRmsAbvGrd']

for element in items_to_remove:
    categorical.remove(element)
    
print(categorical)

['MSSubClass', 'OverallQual', 'OverallCond', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'PoolArea', 'MoSold']


In [8]:
features = train_set.copy().astype(ncf_dict)
labels = features.pop(label)

In [9]:
features_dict = {name: np.array(value) 
                         for name, value in features.items()}

In [10]:
def preprocess_data(data_frame):    
    inputs = {}

    for name, column in data_frame.items():
        dtype = column.dtype
        if dtype == object:
            dtype = tf.string
        else:
            dtype = tf.float32
        inputs[name] = tf.keras.Input(shape=(1,), name=name, dtype=dtype)
        
    numeric_inputs = {name:input for name, input in inputs.items() 
                      if input.dtype==tf.float32}


    x = layers.Concatenate()(list(numeric_inputs.values()))
    norm = layers.Normalization()
    norm.adapt(np.array(train_set[numeric_inputs.keys()]))
    all_numeric_inputs = norm(x)

    preprocessed_inputs = [all_numeric_inputs]
    
    for name, input in inputs.items():
        if input.dtype == tf.float32 and name not in categorical:
            continue 
            
        if name in categorical:
            data_frame[name] = data_frame[name].astype(str)
        
        lookup = tf.keras.layers.StringLookup(vocabulary=np.unique(data_frame[name]))
        
            
        one_hot = layers.CategoryEncoding(num_tokens=lookup.vocabulary_size())
        x = lookup(input)
        x = one_hot(x)
        preprocessed_inputs.append(x)
    
    preprocessed_inputs_cat = layers.Concatenate()(preprocessed_inputs)

    preprocessing = tf.keras.Model(inputs, preprocessed_inputs_cat)
    
    return [inputs, preprocessing]

In [51]:
def model(data_frame):
    body = tf.keras.Sequential([
    layers.Dropout(0.2),
    layers.Dense(128, activation='LeakyReLU'),
    layers.Dense(1)
    ])
    
    inputs, preprocessing_head = preprocess_data(data_frame)
    preprocessed_inputs = preprocessing_head(inputs)
    result = body(preprocessed_inputs)
    model = tf.keras.Model(inputs, result)
    

    model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam())
    return model

In [52]:
model = model(features)

SyntaxError: invalid syntax (3871032513.py, line 3)

In [50]:
model.fit(x=features_dict, y=labels, validation_split = 0.2, epochs=500, callbacks=[LearningRateScheduler(lr_time_based_decay, verbose=1)])

Epoch 1/500
37/37 [==============================] - 2s 20ms/step - loss: 180579.5469 - val_loss: 182220.7344
Epoch 2/500
37/37 [==============================] - 0s 3ms/step - loss: 180544.8438 - val_loss: 182174.2812
Epoch 3/500
37/37 [==============================] - 0s 3ms/step - loss: 180482.5312 - val_loss: 182097.6250
Epoch 4/500
37/37 [==============================] - 0s 3ms/step - loss: 180387.7656 - val_loss: 181988.1562
Epoch 5/500
37/37 [==============================] - 0s 3ms/step - loss: 180258.8750 - val_loss: 181845.0625
Epoch 6/500
37/37 [==============================] - 0s 3ms/step - loss: 180091.2344 - val_loss: 181666.7812
Epoch 7/500
37/37 [==============================] - 0s 3ms/step - loss: 179892.4844 - val_loss: 181454.8594
Epoch 8/500
37/37 [==============================] - 0s 3ms/step - loss: 179658.3281 - val_loss: 181209.2969
Epoch 9/500
37/37 [==============================] - 0s 3ms/step - loss: 179389.8438 - val_loss: 180930.1875
Epoch 10/500
37/37

37/37 [==============================] - 0s 3ms/step - loss: 103291.2891 - val_loss: 106308.7969
Epoch 76/500
37/37 [==============================] - 0s 3ms/step - loss: 101761.7188 - val_loss: 104604.7500
Epoch 77/500
37/37 [==============================] - 0s 3ms/step - loss: 99895.7578 - val_loss: 102901.2578
Epoch 78/500
37/37 [==============================] - 0s 3ms/step - loss: 98271.9688 - val_loss: 101188.7656
Epoch 79/500
37/37 [==============================] - 0s 3ms/step - loss: 96641.0781 - val_loss: 99484.3359
Epoch 80/500
37/37 [==============================] - 0s 3ms/step - loss: 94453.3047 - val_loss: 97791.2109
Epoch 81/500
37/37 [==============================] - 0s 3ms/step - loss: 92644.7266 - val_loss: 96104.1953
Epoch 82/500
37/37 [==============================] - 0s 3ms/step - loss: 91317.0938 - val_loss: 94439.9531
Epoch 83/500
37/37 [==============================] - 0s 3ms/step - loss: 89908.5078 - val_loss: 92771.5859
Epoch 84/500
37/37 [===============

37/37 [==============================] - 0s 3ms/step - loss: 40962.9766 - val_loss: 40787.6289
Epoch 151/500
37/37 [==============================] - 0s 3ms/step - loss: 40359.7070 - val_loss: 40527.6445
Epoch 152/500
37/37 [==============================] - 0s 3ms/step - loss: 40002.6172 - val_loss: 40296.3281
Epoch 153/500
37/37 [==============================] - 0s 3ms/step - loss: 40392.3477 - val_loss: 40079.7344
Epoch 154/500
37/37 [==============================] - 0s 3ms/step - loss: 40149.6445 - val_loss: 39820.0586
Epoch 155/500
37/37 [==============================] - 0s 3ms/step - loss: 39578.7500 - val_loss: 39568.6250
Epoch 156/500
37/37 [==============================] - 0s 3ms/step - loss: 39669.5703 - val_loss: 39356.3906
Epoch 157/500
37/37 [==============================] - 0s 3ms/step - loss: 39329.7109 - val_loss: 39142.3477
Epoch 158/500
37/37 [==============================] - 0s 3ms/step - loss: 39626.7070 - val_loss: 38914.6953
Epoch 159/500
37/37 [============

37/37 [==============================] - 0s 3ms/step - loss: 29133.7324 - val_loss: 29142.4238
Epoch 226/500
37/37 [==============================] - 0s 3ms/step - loss: 29087.8809 - val_loss: 29081.0215
Epoch 227/500
37/37 [==============================] - 0s 3ms/step - loss: 28955.0586 - val_loss: 28968.1504
Epoch 228/500
37/37 [==============================] - 0s 3ms/step - loss: 29159.5820 - val_loss: 28897.4316
Epoch 229/500
37/37 [==============================] - 0s 3ms/step - loss: 28803.7285 - val_loss: 28841.2148
Epoch 230/500
37/37 [==============================] - 0s 3ms/step - loss: 28564.2637 - val_loss: 28762.2012
Epoch 231/500
37/37 [==============================] - 0s 3ms/step - loss: 29271.3223 - val_loss: 28671.1914
Epoch 232/500
37/37 [==============================] - 0s 3ms/step - loss: 28792.1680 - val_loss: 28628.7422
Epoch 233/500
37/37 [==============================] - 0s 4ms/step - loss: 28344.1406 - val_loss: 28601.4102
Epoch 234/500
37/37 [============

37/37 [==============================] - 0s 3ms/step - loss: 25625.0293 - val_loss: 25560.1582
Epoch 301/500
37/37 [==============================] - 0s 3ms/step - loss: 25033.8828 - val_loss: 25497.8555
Epoch 302/500
37/37 [==============================] - 0s 3ms/step - loss: 25551.8125 - val_loss: 25485.6055
Epoch 303/500
37/37 [==============================] - 0s 3ms/step - loss: 25244.3926 - val_loss: 25513.0254
Epoch 304/500
37/37 [==============================] - 0s 4ms/step - loss: 25754.8613 - val_loss: 25434.7031
Epoch 305/500
37/37 [==============================] - 0s 4ms/step - loss: 25067.6484 - val_loss: 25351.8203
Epoch 306/500
37/37 [==============================] - 0s 3ms/step - loss: 24907.3496 - val_loss: 25362.1973
Epoch 307/500
37/37 [==============================] - 0s 4ms/step - loss: 25317.4785 - val_loss: 25339.8770
Epoch 308/500
37/37 [==============================] - 0s 4ms/step - loss: 25208.7285 - val_loss: 25267.2383
Epoch 309/500
37/37 [============

37/37 [==============================] - 0s 3ms/step - loss: 22707.6406 - val_loss: 23320.3887
Epoch 376/500
37/37 [==============================] - 0s 3ms/step - loss: 23737.7754 - val_loss: 23326.4688
Epoch 377/500
37/37 [==============================] - 0s 3ms/step - loss: 23991.7422 - val_loss: 23304.8281
Epoch 378/500
37/37 [==============================] - 0s 3ms/step - loss: 23819.9199 - val_loss: 23326.3301
Epoch 379/500
37/37 [==============================] - 0s 3ms/step - loss: 23195.2969 - val_loss: 23287.7598
Epoch 380/500
37/37 [==============================] - 0s 3ms/step - loss: 23518.2715 - val_loss: 23258.7637
Epoch 381/500
37/37 [==============================] - 0s 3ms/step - loss: 23219.2949 - val_loss: 23252.9121
Epoch 382/500
37/37 [==============================] - 0s 3ms/step - loss: 23721.2266 - val_loss: 23244.3594
Epoch 383/500
37/37 [==============================] - 0s 3ms/step - loss: 23336.8809 - val_loss: 23212.4453
Epoch 384/500
37/37 [============

37/37 [==============================] - 0s 4ms/step - loss: 22225.7637 - val_loss: 21964.9219
Epoch 451/500
37/37 [==============================] - 0s 3ms/step - loss: 22089.3633 - val_loss: 21940.2910
Epoch 452/500
37/37 [==============================] - 0s 3ms/step - loss: 22421.9512 - val_loss: 21954.2949
Epoch 453/500
37/37 [==============================] - 0s 3ms/step - loss: 22428.2617 - val_loss: 21900.8418
Epoch 454/500
37/37 [==============================] - 0s 3ms/step - loss: 21940.2793 - val_loss: 21974.3438
Epoch 455/500
37/37 [==============================] - 0s 3ms/step - loss: 22503.8398 - val_loss: 21923.8516
Epoch 456/500
37/37 [==============================] - 0s 3ms/step - loss: 21761.2324 - val_loss: 21890.5254
Epoch 457/500
37/37 [==============================] - 0s 3ms/step - loss: 22209.0195 - val_loss: 21886.2090
Epoch 458/500
37/37 [==============================] - 0s 3ms/step - loss: 22110.4121 - val_loss: 21907.1152
Epoch 459/500
37/37 [============

In [34]:
test_features_dict = {name: np.array(value) 
                         for name, value in test_set.copy().items()}
array = model.predict(test_features_dict)

AttributeError: 'function' object has no attribute 'predict'

In [22]:
import csv

with open('regression_solution_7.csv', 'w', newline='') as file:
    fieldnames = ['Id', 'SalePrice']
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    writer.writeheader()
    n = 1461
    for element in array:
        writer.writerow({'Id': n, 'SalePrice': element[0]})
        n += 1